You will need a Google Cloud account and a project set up for Earth Engine. If this is your first time using Google Earth Engine, first visit https://earthengine.google.com and click `Get Started`. Once you have a Google Cloud account and a project registered for Google Earth Engine, you should be able to run this script.

Visit https://developers.google.com/earth-engine/ for User Guides and Developer References.

Dataset available from Google Earth Engine are listed here: https://developers.google.com/earth-engine/datasets/

This script demonstrates how to use the Earth Engine Python package to access Google's Satellite Embeddings dataset (https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_SATELLITE_EMBEDDING_V1_ANNUAL)

1. Load dependencies, authenticate, and create a project

In [ ]:
import ee

ee.Authenticate()

ee.Initialize()

2. Read locations from file

In [ ]:
import csv

locations = list(csv.DictReader(open('assets/locations.csv')))
print(f'Loaded {len(locations)} locations')
for loc in locations[:3]:
    print(loc)

3. Create a function to extract embeddings for a particular location and date range

In [ ]:
def extract_embedding(latitude, longitude, start_date, end_date):
    lat = float(latitude)
    lon = float(longitude)
    point = ee.Geometry.Point([lon, lat])

    collection = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL') \
        .filterDate(start_date, end_date) \
        .filterBounds(point)

    return collection.mean().sample(region=point, scale=30).first().toDictionary()

Earth Engine datasets are categorized as `Image`s (raster data) and `Feature`s (a geometry with attributes), and organized into `ImageCollection`s and `FeatureCollection`s. See [Objects and Methods Overview](https://developers.google.com/earth-engine/guides/objects_methods_overview) for more details.

Here, we start from the Google Satellite Embeddings V1 product. We use `filterDate()` and `filterBounds()` to select data for a single point over a range of dates. Calling `mean()` on the `ImageCollection` returns an `Image` with mean values for each band. We then use `sample()` to return a subset of the pixels in the `Image` as a `FeatureCollection` (in this case, the pixel that corresponds to our point location). Finally, we select the first (of one) feature (with `first()`) in the collection and convert it into a set of key-value pairs (with `toDictionary()`).

See the [API Reference](https://developers.google.com/earth-engine/apidocs) for more details and example scripts.

4. Extract embeddings for each location for one year

In [ ]:
for location in locations:
    try:
        embedding = extract_embedding(location['lat'], location['lon'], '2022-01-01', '2022-12-31')
        print(f"Location ({location['lat']}, {location['lon']}) embedding: {embedding.getInfo()}")
    except Exception as e:
        print(f"Error processing location ({location['lat']}, {location['lon']}): {e}")